In [1]:
!pip install -q PyPDF2
!pip install -q tiktoken   # BPE
!pip install -q groq
!pip install -q -U transformers


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from PyPDF2 import PdfReader
import tiktoken
import os
from groq import Groq
import groq
import glob
import time
from transformers import pipeline

In [3]:
# Extracting Text From pages
def read_pdf(file_path):
    reader = PdfReader(file_path)
    for page in reader.pages:
        yield page.extract_text()

# Counting the number of tokens
def number_of_tokens(text):
  tokenizer = tiktoken.get_encoding("gpt2")
  ids = tokenizer.encode(text)
  return len(ids)

# Function to summarize text
def summarize_text(text, chunk_size=500,device=0):
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6",device=device)

    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    summaries = []
    for chunk in chunks:
        max_length = min(150,max(20,number_of_tokens(chunk) // 2))
        min_length = min(50,max(10,number_of_tokens(chunk) // 2))
        summary = summarizer(chunk, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
        summaries.append(summary)

    combined_summary = " ".join(summaries)
    return combined_summary

In [4]:
import os

# Set the environment variable
os.environ['GROQ_API_KEY'] = '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'

In [5]:
# Initialize Groq client
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [6]:
# Function to generate Q&A pairs from text using Groq's API
def generate_questions_and_answers(text,text_summarization, model="llama-3.1-70b-versatile"):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"""Generate questions and answers from the given text. 
                The answers should be based on what's in the text, not on your knowledge. 
                Follow this model in generating: 'Q: ...\nA: ...'. 
                Here is the text: {text}. 
                Also, consider the summarization of the previous text to help 
                in generating questions and answers: {text_summarization}."""
            }
        ],
        model=model,
    )

    # Extracting the content of the response
    if response and response.choices:
        return response.choices[0].message.content
    else:
        return "No Q&A generated."

In [7]:
pdf_files = glob.glob('Book_?.pdf')

def Generate():
    Number_of_Tokens_per_minute = 0
    Number_of_Request_per_minute = 0
    Number_of_Tokens_per_day = 0
    Number_of_Request_per_day = 0
    for i,pdf in enumerate(pdf_files):
        output_file = f'Book_{i+1}_q&a.txt'
        text_summarization = ""
        print(f"Book {i+1}")
        with open(output_file, 'w', encoding='utf-8') as file:
            for j,page_text in enumerate(read_pdf(pdf)):

                current_page_tokens = number_of_tokens(page_text)
                current_summary_tokens = number_of_tokens(text_summarization)
                
                Number_of_Tokens_per_minute += (current_page_tokens + current_summary_tokens)
                Number_of_Tokens_per_day += (current_page_tokens + current_summary_tokens)
                
                if(Number_of_Tokens_per_minute > 130000 or Number_of_Request_per_minute >= 100):
                    Number_of_Tokens_per_minute = 0
                    Number_of_Request_per_minute = 0
                    print("please just wait 1 min ....")
                    time.sleep(65)
                    
                if(Number_of_Tokens_per_day >= 1000000 or Number_of_Request_per_day >= 14400):
                    Number_of_Tokens_per_day = 0
                    Number_of_Request_per_day = 0
                    Number_of_Tokens_per_minute = 0
                    Number_of_Request_per_minute = 0
                    print("please you have to wait 1 day ....")
                    time.sleep(86100)
                    
                q_and_a = generate_questions_and_answers(page_text,text_summarization)
                current_q_and_a_tokens = number_of_tokens(q_and_a)

                Number_of_Tokens_per_minute += current_q_and_a_tokens
                Number_of_Tokens_per_day += current_q_and_a_tokens
                
                Number_of_Request_per_minute += 1
                Number_of_Request_per_day += 1
                text_summarization += summarize_text(page_text)
                
                file.write(f"Page {j+1} Q&A:\n{q_and_a}\n\n")
                if((j+1)%10==0 or j==0):
                    print(f"-> Page {j+1} converted\n-> ...")
            
            print(f"-> Q&A generation for pdf {i+1} completed.")
            print('\n')

In [ ]:
Generate()

Book 1
-> Page 1 converted
-> ...
-> Page 10 converted
-> ...
-> Page 20 converted
-> ...


Your max_length is set to 20, but your input_length is only 4. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)


please just wait 1 min ....


Your max_length is set to 20, but your input_length is only 13. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Your max_length is set to 20, but your input_length is only 4. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)


-> Q&A generation for pdf 1 completed.


Book 2
-> Page 1 converted
-> ...


...

-> Page 10 converted
-> ...


Your max_length is set to 20, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


-> Page 20 converted
-> ...


Your max_length is set to 20, but your input_length is only 13. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Your max_length is set to 20, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 20, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


-> Page 30 converted
-> ...
